In [11]:
!pip install bs4 requests_html

  Using cached bs4-0.0.2-py2.py3-none-any.whl.metadata (411 bytes)
  Using cached requests_html-0.10.0-py3-none-any.whl.metadata (15 kB)
     ---------------------------------------- 0.0/49.3 kB ? eta -:--:--
     ---------------------------------------- 49.3/49.3 kB ? eta 0:00:00
   ---------------------------------------- 0.0/82.9 kB ? eta -:--:--
   ---------------------------------------- 82.9/82.9 kB ? eta 0:00:00
   ---------------------------------------- 0.0/143.9 kB ? eta -:--:--
   ---------------------------------------- 143.9/143.9 kB 8.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/101.4 kB ? eta -:--:--
   ---------------------------------------- 101.4/101.4 kB ? eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7


In [61]:
import requests
from bs4 import BeautifulSoup 
import re
from requests_html import HTMLSession
import numpy as np
import pandas as pd
import random
import unicodedata
import codecs
prefs = open('japanprefs.txt', 'rb').readlines()

In [62]:
prefs = [x.decode("utf-8", "replace").replace('\t', ' ').replace('\r\n', '').split(' ')[1:] for x in prefs]

In [63]:
d = dict(zip([x[1] for x in prefs], [x[0] for x in prefs]))
keys = list(d.keys())

In [64]:
random.shuffle(keys)
s = 0
t = 0
for p in keys:
    inp = input(d[p] + ': ')
    if inp.lower().strip() == p:
        print('Correct!')
        s += 1
    else:
        print('Incorrect. This is', p)
    t += 1
    print(('0' if s < 10 else '') + str(s) + '/' + ('0' if t < 10 else '') + str(t), '(' + str(round(s/t*100, 1)) + '%)\n')

茨城県:  s


Incorrect. This is ibaraki
00/01 (0.0%)



高知県:  s


Incorrect. This is kochi
00/02 (0.0%)



島根県:  d


...

東京都:  kyoto


Incorrect. This is tokyo
00/05 (0.0%)



福岡県:  fukuoka


Correct!
01/06 (16.7%)



KeyboardInterrupt: Interrupted by user

In [65]:
chd = {}
for p in d.keys():
    for ch in d[p]:
        if ch in chd:
            chd[ch] = chd[ch] + [p]
        else:
            chd[ch] = [p]

In [66]:
for ch in chd:
    if len(chd[ch]) > 1 and ch != '県':
        print(ch, [x + '|' + d[x] for x in chd[ch]])

宮 ['miyagi|宮城県', 'miyazaki|宮崎県']
城 ['miyagi|宮城県', 'ibaraki|茨城県']
山 ['yamagata|山形県', 'toyama|富山県', 'yamanashi|山梨県', 'wakayama|和歌山県', 'okayama|岡山県', 'yamaguchi|山口県']
福 ['fukushima|福島県', 'fukui|福井県', 'fukuoka|福岡県']
島 ['fukushima|福島県', 'shimane|島根県', 'hiroshima|広島県', 'tokushima|徳島県', 'kagoshima|鹿児島県']
京 ['tokyo|東京都', 'kyoto|京都府']
都 ['tokyo|東京都', 'kyoto|京都府']
奈 ['kanagawa|神奈川県', 'nara|奈良県']
川 ['kanagawa|神奈川県', 'ishikawa|石川県', 'kagawa|香川県']
長 ['nagano|長野県', 'nagasaki|長崎県']
岡 ['shizuoka|静岡県', 'okayama|岡山県', 'fukuoka|福岡県']
愛 ['aichi|愛知県', 'ehime|愛媛県']
知 ['aichi|愛知県', 'kochi|高知県']
賀 ['shiga|滋賀県', 'saga|佐賀県']
府 ['kyoto|京都府', 'osaka|大阪府']
大 ['osaka|大阪府', 'oita|大分県']
崎 ['nagasaki|長崎県', 'miyazaki|宮崎県']


In [67]:
#jisho
pieces = ['meanings english sense', 'kun readings', 'on readings', 'info clearfix']
df = pd.DataFrame(index=list(chd.keys()), columns=pieces + ['names'])
df['names'] = list(chd.values())

In [68]:
s = 0
for ch in chd.keys():
    s += 1
    url = "https://jisho.org/search/" + ch
    session = HTMLSession()
    response = session.get(url)
    html = response.html.raw_html
    soup = BeautifulSoup(html, 'html5lib')

    entry = soup.find('div', {'class': 'entry'})
    print(s, ch, [x + '|' + d[x] for x in chd[ch]])
    for piece in pieces:
        piec = soup.find('div', {'class': piece})
        if piec:
            df.loc[ch, piece] = piec.text.replace('\n', '').replace('\t', '').replace('   ', '').strip()
#                 print(piec.text.replace('     ', ''))
#         print('\n')


1 北 ['hokkaido|北海道']
2 海 ['hokkaido|北海道']
3 道 ['hokkaido|北海道']
4 青 ['aomori|青森県']
5 森 ['aomori|青森県']
6 県 ['aomori|青森県', 'iwate|岩手県', 'miyagi|宮城県', 'akita|秋田県', 'yamagata|山形県', 'fukushima|福島県', 'ibaraki|茨城県', 'tochigi|栃木県', 'gunma|群馬県', 'saitama|埼玉県', 'chiba|千葉県', 'kanagawa|神奈川県', 'niigata|新潟県', 'toyama|富山県', 'ishikawa|石川県', 'fukui|福井県', 'yamanashi|山梨県', 'nagano|長野県', 'gifu|岐阜県', 'shizuoka|静岡県', 'aichi|愛知県', 'mie|三重県', 'shiga|滋賀県', 'hyogo|兵庫県', 'nara|奈良県', 'wakayama|和歌山県', 'tottori|鳥取県', 'shimane|島根県', 'okayama|岡山県', 'hiroshima|広島県', 'yamaguchi|山口県', 'tokushima|徳島県', 'kagawa|香川県', 'ehime|愛媛県', 'kochi|高知県', 'fukuoka|福岡県', 'saga|佐賀県', 'nagasaki|長崎県', 'kumamoto|熊本県', 'oita|大分県', 'miyazaki|宮崎県', 'kagoshima|鹿児島県', 'okinawa|沖縄県']
7 岩 ['iwate|岩手県']
8 手 ['iwate|岩手県']
9 宮 ['miyagi|宮城県', 'miyazaki|宮崎県']
10 城 ['miyagi|宮城県', 'ibaraki|茨城県']
11 秋 ['akita|秋田県']
12 田 ['akita|秋田県']
13 山 ['yamagata|山形県', 'toyama|富山県', 'yamanashi|山梨県', 'wakayama|和歌山県', 'okayama|岡山県', 'yamaguchi|山口県']
14 形 ['yamagata|山形県']

In [47]:
df

,meanings english sense,kun readings,on readings,info clearfix,names
北,north,Kun: きた,On: ホク,"5 strokes. JLPT N5. Jōyō kanji, taught in ...",[hokkaido]
海,"sea, ocean",Kun: うみ,On: カイ,"9 strokes. JLPT N4. Jōyō kanji, taught in ...",[hokkaido]
道,"road-way, street, district, journey, course, m...",Kun: みち、 いう,On: ドウ、 トウ,"12 strokes. JLPT N4. Jōyō kanji, taught in...",[hokkaido]
青,"blue, green",Kun: あお、 あお-、 あお.い,On: セイ、 ショウ,"8 strokes. JLPT N4. Jōyō kanji, taught in ...",[aomori]
森,"forest, woods",Kun: もり,On: シン,"12 strokes. JLPT N2. Jōyō kanji, taught in...",[aomori]
...,...,...,...,...,...
分,"part, minute of time, segment, share, degree, ...",Kun: わ.ける、 わ.け、 わ.かれる、 わ.かる、 わ.かつ,On: ブン、 フン、 ブ,"4 strokes. Jōyō kanji, taught in grade 2.",[oita]
鹿,deer,Kun: しか、 か,On: ロク,"11 strokes. JLPT N1. Jōyō kanji, taught in...",[kagoshima]
児,"newborn babe, child, young of animals",Kun: こ、 -こ、 -っこ,On: ジ、 ニ、 ゲイ,"7 strokes. JLPT N2. Jōyō kanji, taught in ...",[kagoshima]
沖,"open sea, offing, rise high into sky",Kun: おき、 おきつ、 ちゅう.する、 わく,On: チュウ,"7 strokes. JLPT N1. Jōyō kanji, taught in ...",[okinawa]


In [51]:
df['level'] = df['info clearfix'].str.extract(r'JLPT N(\w)',  
                                     expand = True) 
df['grade'] = df['info clearfix'].str.extract(r'taught in grade (\w)',  
                                     expand = True) 

df.sort_values(by=['level', 'grade'], ascending=[False, True])

df2 = df.sort_values(by=['level', 'grade'], ascending=[False, True])
df2.to_csv('japaneseprefkanji.csv')
df2.to_json('japaneseprefkanji.json')
df2

,meanings english sense,kun readings,on readings,info clearfix,names,level,grade
山,mountain,Kun: やま,On: サン、 セン,"3 strokes. JLPT N5. Jōyō kanji, taught in ...","[yamagata, toyama, yamanashi, wakayama, okayam...",5,1
木,"tree, wood",Kun: き、 こ-,On: ボク、 モク,"4 strokes. JLPT N5. Jōyō kanji, taught in ...",[tochigi],5,1
千,thousand,Kun: ち,On: セン,"3 strokes. JLPT N5. Jōyō kanji, taught in ...",[chiba],5,1
川,"stream, river, river or three-stroke river rad...",Kun: かわ,On: セン,"3 strokes. JLPT N5. Jōyō kanji, taught in ...","[kanagawa, ishikawa, kagawa]",5,1
三,three,Kun: み、 み.つ、 みっ.つ,On: サン、 ゾウ,"3 strokes. JLPT N5. Jōyō kanji, taught in ...",[mie],5,1
...,...,...,...,...,...,...,...
栃,"horse chestnut, (kokuji)",Kun: とち,NaN,"9 strokes. Jōyō kanji, taught in grade 4.",[tochigi],NaN,4
埼,"cape, spit, promontory",Kun: さき、 さい、 みさき,On: キ,"11 strokes. Jōyō kanji, taught in grade 4.",[saitama],NaN,4
阜,"hill, mound, left village radical (no. 170)",NaN,On: フ、 フウ,"8 strokes. Jōyō kanji, taught in grade 4.",[gifu],NaN,4
岡,"mount, hill, knoll",Kun: おか,On: コウ,"8 strokes. Jōyō kanji, taught in grade 4.","[shizuoka, okayama, fukuoka]",NaN,4


In [100]:
for i, r in df.iterrows():
    print(i, ':', r['meanings english sense'])
    print(r['kun readings'])
    print(r['on readings'])
    print('Used in', ', '.join([x + '|' + d[x] for x in r['names']]))
    print()

北 : north
Kun: きた
On: ホク
Used in hokkaido|北海道

海 : sea, ocean
Kun: うみ
On: カイ
Used in hokkaido|北海道

道 : road-way, street, district, journey, course, moral, teachings
Kun: みち、 いう
On: ドウ、 トウ
Used in hokkaido|北海道

青 : blue, green
Kun: あお、 あお-、 あお.い
On: セイ、 ショウ
Used in aomori|青森県

森 : forest, woods
Kun: もり
On: シン
Used in aomori|青森県

県 : prefecture
Kun: か.ける
On: ケン
Used in aomori|青森県, iwate|岩手県, miyagi|宮城県, akita|秋田県, yamagata|山形県, fukushima|福島県, ibaraki|茨城県, tochigi|栃木県, gunma|群馬県, saitama|埼玉県, chiba|千葉県, kanagawa|神奈川県, niigata|新潟県, toyama|富山県, ishikawa|石川県, fukui|福井県, yamanashi|山梨県, nagano|長野県, gifu|岐阜県, shizuoka|静岡県, aichi|愛知県, mie|三重県, shiga|滋賀県, hyogo|兵庫県, nara|奈良県, wakayama|和歌山県, tottori|鳥取県, shimane|島根県, okayama|岡山県, hiroshima|広島県, yamaguchi|山口県, tokushima|徳島県, kagawa|香川県, ehime|愛媛県, kochi|高知県, fukuoka|福岡県, saga|佐賀県, nagasaki|長崎県, kumamoto|熊本県, oita|大分県, miyazaki|宮崎県, kagoshima|鹿児島県, okinawa|沖縄県

岩 : boulder, rock, cliff
Kun: いわ
On: ガン
Used in iwate|岩手県

手 : hand
Kun: て、 て-、 -て、 た-
On

In [105]:
# playground for getting hiragana conversion of kun/on
names = []
for i in range(0x110000): 
    character = chr(i) 
    name = unicodedata.name(character, "") 
    names.append(name)

In [125]:
for i, r in df.iterrows():
    if type(r['kun readings']) == str:
            readings = r['kun readings'].split(' ')[1:]
            for reading in readings:
                rtext = ''
                for ch in reading:
                    cname = unicodedata.name(ch, "")
                    if cname not in ['IDEOGRAPHIC COMMA', 'FULL STOP']:
                        if cname == 'HYPHEN-MINUS':
                            rtext += '-'
                        else:
                            rtext += cname.split(' ')[-1].lower()
                print(rtext)

kita
umi
miti
iu
ao
ao-
aoi
mori
kakeru
iwa
te
te-
-te
ta-
miya
siro
aki
toki
ta
yama
kata
-gata
katati
nari
sima
ibara
kaya
kusabuki
toti
ki
ko-
mureru
mure
mura
muragaru
uma
uma-
ma
saki
sai
misaki
tama
tama-
-dama
ti
ha
higasi
miyako
miyako
kami
kan-
kou-
ikan
karanasi
kawa
atarasii
arata
ara-
nii-
kata
-gata
tomu
tomi
isi
i
nasi
nagai
osa
no
no-
sizu-
sizuka
sizumaru
sizumeru
oka
itosii
kanasii
mederu
osimu
mana
siru
siraseru
mi
mitu
mitutu
e
omoi
omori
omonau
kasaneru
kasanaru
omo
oo-
ookii
-ooini
saka
tuwamono
kura
yoi
-yoi
ii
-ii
yawaragu
yawarageru
nagomu
nagoyaka
aeru
uta
utau
tori
toru
tori
tori-
tori
-dori
ne
-ne
hiroi
hiromaru
hiromeru
hirogaru
hirogeru
kuti
ka
kaori
kaoru
hime
takai
taka
-daka
takamaru
takameru
saki
sai
misaki
kuma
moto
wakeru
wake
wakareru
wakaru
wakatu
sika
ka
ko
-ko
-tuko
oki
okitu
tiyuusuru
waku
nawa
tadasu


# List of Japanese cities (wikipedia)

In [74]:
url = "https://en.wikipedia.org/wiki/List_of_cities_in_Japan"
session = HTMLSession()
response = session.get(url)
html = response.html.raw_html
soup = BeautifulSoup(html, 'html5lib')

table = soup.findAll('table')[2]
cities = pd.read_html(url)[2]

In [75]:
cities.columns

Index(['City (Special Ward)', 'Japanese', 'Prefecture', 'Population',
       'Area (km2)', 'Density (per km2)', 'Founded', 'Website'],
      dtype='object')

In [77]:
cities['Japanese']

0      名古屋市
1       豊橋市
2       岡崎市
3       一宮市
4       瀬戸市
       ... 
811     北杜市
812    上野原市
813     山梨市
814     甲州市
815     中央市
Name: Japanese, Length: 816, dtype: object

In [78]:
d = dict(zip(cities['City (Special Ward)'], cities['Japanese']))
keys = list(d.keys())

chd = {}
for p in d.keys():
    for ch in d[p]:
        if ch in chd:
            chd[ch] = chd[ch] + [p]
        else:
            chd[ch] = [p]
            
            
len(chd)

544

In [79]:
keystodo = [x for x in chd.keys() if 'HIRAGANA' not in unicodedata.name(x) and 'KATAKANA' not in unicodedata.name(x)]
print(len(keystodo))

505


In [80]:
#jisho
pieces = ['meanings english sense', 'kun readings', 'on readings', 'info clearfix']
df = pd.DataFrame(index=keystodo, columns=pieces + ['names'])
df['names'] = list([chd[k] for k in keystodo])

s = 0
for ch in keystodo:
    s += 1
    url = "https://jisho.org/search/" + ch
    session = HTMLSession()
    response = session.get(url)
    html = response.html.raw_html
    soup = BeautifulSoup(html, 'html5lib')

    entry = soup.find('div', {'class': 'entry'})
    print(s, ch, [x + '|' + d[x] for x in chd[ch]])
    for piece in pieces:
        piec = soup.find('div', {'class': piece})
        if piec:
            df.loc[ch, piece] = piec.text.replace('\n', '').replace('\t', '').replace('   ', '').strip()
#                 print(piec.text.replace('     ', ''))
#         print('\n')



1 名 ['Nagoya|名古屋市', 'Kitanagoya|北名古屋市', 'Nayoro|名寄市', 'Ebina|海老名市', 'Tamana|玉名市', 'Kuwana|桑名市', 'Nabari|名張市', 'Natori|名取市', 'Nago|名護市']
2 古 ['Nagoya|名古屋市', 'Kitanagoya|北名古屋市', 'Koga|古河市', 'Kakogawa|加古川市', 'Miyako|宮古市', 'Miyakojima|宮古島市']
3 屋 ['Nagoya|名古屋市', 'Kitanagoya|北名古屋市', 'Ashiya|芦屋市', 'Kanoya|鹿屋市', 'Neyagawa|寝屋川市']
4 市 ['Nagoya|名古屋市', 'Toyohashi|豊橋市', 'Okazaki|岡崎市', 'Ichinomiya|一宮市', 'Seto|瀬戸市', 'Handa|半田市', 'Kasugai|春日井市', 'Toyokawa|豊川市', 'Tsushima|対馬市', 'Hekinan|碧南市', 'Kariya|刈谷市', 'Toyota|豊田市', 'Anjō|安城市', 'Nishio|西尾市', 'Gamagōri|蒲郡市', 'Inuyama|犬山市', 'Tokoname|常滑市', 'Kōnan|香南市', 'Komaki|小牧市', 'Inazawa|稲沢市', 'Tōkai|東海市', 'Ōbu|大府市', 'Chita|知多市', 'Chiryū|知立市', 'Owariasahi|尾張旭市', 'Takahama|高浜市', 'Iwakura|岩倉市', 'Toyoake|豊明市', 'Nisshin|日進市', 'Tahara|田原市', 'Aisai|愛西市', 'Kiyosu|清須市', 'Shinshiro|新城市', 'Kitanagoya|北名古屋市', 'Yatomi|弥富市', 'Miyoshi|三好市', 'Ama|あま市', 'Nagakute|長久手市', 'Akita|秋田市', 'Ōdate|大館市', 'Kazuno|鹿角市', 'Daisen|大仙市', 'Katagami|潟上市', 'Kitaakita|北秋田市', 'Oga|男鹿市', 'Yurihonjō|

Traceback (most recent call last):
  File "C:\Users\dippe\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\dippe\AppData\Local\Temp\ipykernel_12348\4109340912.py", line 11, in <module>
    response = session.get(url)
               ^^^^^^^^^^^^^^^^
  File "C:\Users\dippe\anaconda3\Lib\site-packages\requests\sessions.py", line 602, in get
    return self.request("GET", url, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\dippe\anaconda3\Lib\site-packages\requests\sessions.py", line 589, in request
    resp = self.send(prep, **send_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\dippe\anaconda3\Lib\site-packages\requests\sessions.py", line 703, in send
    r = adapter.send(request, **kwargs)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\dippe\anaconda3\Lib\site-packages\requests\adapters.py", line 486, in send
    resp = conn.u

330 姶 ['Aira|姶良市']
331 茅 ['Chigasaki|茅ヶ崎市', 'Chino|茅野市']
332 逗 ['Zushi|逗子市']
333 模 ['Sagamihara|相模原市']
334 秦 ['Hadano|秦野市']
335 厚 ['Atsugi|厚木市']
336 老 ['Ebina|海老名市']
337 座 ['Zama|座間市', 'Niiza|新座市']
338 足 ['Minamiashigara|南足柄市', 'Ashikaga|足利市', 'Adachi|足立区']
339 柄 ['Minamiashigara|南足柄市']
340 綾 ['Ayase|綾瀬市', 'Ayabe|綾部市']
341 毛 ['Sukumo|宿毛市']
342 万 ['Shimanto|四万十市', 'Imari|伊万里市']
343 熊 ['Kumamoto|熊本市', 'Kumano|熊野市', 'Kumagaya|熊谷市']
344 人 ['Hitoyoshi|人吉市']
345 吉 ['Hitoyoshi|人吉市', 'Yoshikawa|吉川市', 'Yoshinogawa|吉野川市', 'Kurayoshi|倉吉市', 'Fujiyoshida|富士吉田市']
346 荒 ['Arao|荒尾市', 'Arakawa|荒川区']
347 俣 ['Minamata|水俣市']
348 菊 ['Kikuchi|菊池市', 'Kikugawa|菊川市']
349 池 ['Kikuchi|菊池市', 'Ikeda|池田市']
350 天 ['Kami-Amakusa|上天草市', 'Amakusa|天草市', 'Tenri|天理市', 'Tendō|天童市']
351 草 ['Kami-Amakusa|上天草市', 'Amakusa|天草市', 'Sōka|草加市', 'Kusatsu|草津市']
352 蘇 ['Aso|阿蘇市']
353 合 ['Kōshi|合志市']
354 京 ['Kyoto|京都市', 'Nagaokakyō|長岡京市', 'Kyōtanabe|京田辺市', 'Kyōtango|京丹後市', 'Nishitōkyō|西東京市', 'Tokyo|東京特別区部', 'Bunkyō|文京区']
355 都 ['Kyoto|

In [60]:
df['level'] = df['info clearfix'].str.extract(r'JLPT N(\w)',  
                                     expand = True) 
df['grade'] = df['info clearfix'].str.extract(r'taught in grade (\w)',  
                                     expand = True) 

df.sort_values(by=['level', 'grade'], ascending=[False, True])

df2 = df.sort_values(by=['level', 'grade'], ascending=[False, True])
df2.to_csv('japanesecitykanji.csv')
df2.to_json('japanesecitykanji.json')
df2

,meanings english sense,kun readings,on readings,info clearfix,names,level,grade
名,"name, noted, distinguished, reputation",Kun: な、 -な,On: メイ、 ミョウ,"6 strokes. JLPT N5. Jōyō kanji, taught in ...","[Nagoya, Kitanagoya, Nayoro, Ebina, Tamana, Ku...",5,1
一,"one, one radical (no.1)",Kun: ひと-、 ひと.つ,On: イチ、 イツ,"1 stroke. JLPT N5. Jōyō kanji, taught in g...","[Ichinomiya, Ichinoseki]",5,1
日,"day, sun, Japan, counter for days",Kun: ひ、 -び、 -か,On: ニチ、 ジツ,"4 strokes. JLPT N5. Jōyō kanji, taught in ...","[Kasugai, Nisshin, Kasuga, Hatsukaichi, Hitach...",5,1
川,"stream, river, river or three-stroke river rad...",Kun: かわ,On: セン,"3 strokes. JLPT N5. Jōyō kanji, taught in ...","[Toyokawa, Goshogawara, Hirakawa, Ichikawa, Ka...",5,1
山,mountain,Kun: やま,On: サン、 セン,"3 strokes. JLPT N5. Jōyō kanji, taught in ...","[Inuyama, Tateyama, Nagareyama, Sanmu, Matsuya...",5,1
...,...,...,...,...,...,...,...
狛,"archaic part of Korea, lion-dog shrine guards",Kun: こま,On: ハク,8 strokes.,[Komae],NaN,NaN
砺,"whetstone, polish",Kun: あらと、 みが.く,On: レイ,10 strokes.,"[Tonami, Nanto]",NaN,NaN
坊,"boy, priest's residence, priest",NaN,On: ボウ、 ボッ,"7 strokes. Jōyō kanji, taught in junior high.",[Gobō],NaN,NaN
祢,ancestral shrine,NaN,On: ネ、 デイ、 ナイ,9 strokes. Jinmeiyō kanji.,[Mine],NaN,NaN


In [59]:
for i, r in df2.iterrows():
    print(i, ':', r['meanings english sense'])
    print(r['kun readings'])
    print(r['on readings'])
    print(r['info clearfix'])
    print('Used in', ', '.join([x + '|' + d[x] for x in r['names']]))
    print('\n')

名 : name, noted, distinguished, reputation
Kun: な、 -な
On: メイ、 ミョウ
6 strokes.  JLPT N5.  Jōyō kanji, taught in grade 1.
Used in Nagoya|名古屋市, Kitanagoya|北名古屋市, Nayoro|名寄市, Ebina|海老名市, Tamana|玉名市, Kuwana|桑名市, Nabari|名張市, Natori|名取市, Nago|名護市


一 : one, one radical (no.1)
Kun: ひと-、 ひと.つ
On: イチ、 イツ
1 stroke.  JLPT N5.  Jōyō kanji, taught in grade 1.
Used in Ichinomiya|一宮市, Ichinoseki|一関市


日 : day, sun, Japan, counter for days
Kun: ひ、 -び、 -か
On: ニチ、 ジツ
4 strokes.  JLPT N5.  Jōyō kanji, taught in grade 1.
Used in Kasugai|春日井市, Nisshin|日進市, Kasuga|春日市, Hatsukaichi|廿日市市, Hitachi|日立市, Hioki|日置市, Mukō|向日市, Yokkaichi|四日市市, Nichinan|日南市, Hyūga|日向市, Tōkamachi|十日町市, Hita|日田市, Hidaka|日高市, Kasukabe|春日部市, Nikkō|日光市, Hino|日野市


川 : stream, river, river or three-stroke river radical (no. 47)
Kun: かわ
On: セン
3 strokes.  JLPT N5.  Jōyō kanji, taught in grade 1.
Used in Toyokawa|豊川市, Goshogawara|五所川原市, Hirakawa|平川市, Ichikawa|市川市, Kamogawa|鴨川市, Tagawa|田川市, Yanagawa|柳川市, Ōkawa|大川市, Nakagawa|那珂川市, Sukag

In [15]:
df2['level'].value_counts().sort_index(ascending=False)

5     52
4     57
3     91
2     82
1    147
Name: level, dtype: int64

In [16]:
df2['grade'].value_counts().sort_index()

1    61
2    73
3    66
4    65
5    32
6    23
Name: grade, dtype: int64

,meanings english sense,kun readings,on readings,info clearfix,names
名,"name, noted, distinguished, reputation",Kun: な、 -な,On: メイ、 ミョウ,"6 strokes. JLPT N5. Jōyō kanji, taught in ...","[Nagoya, Kitanagoya, Nayoro, Ebina, Tamana, Ku..."
古,old,Kun: ふる.い、 ふる-、 -ふる.す,On: コ,"5 strokes. JLPT N4. Jōyō kanji, taught in ...","[Nagoya, Kitanagoya, Koga, Kakogawa, Miyako, M..."
屋,"roof, house, shop, dealer, seller",Kun: や,On: オク,"9 strokes. JLPT N4. Jōyō kanji, taught in ...","[Nagoya, Kitanagoya, Ashiya, Kanoya, Neyagawa]"
市,"market, city, town",Kun: いち,On: シ,"5 strokes. JLPT N3. Jōyō kanji, taught in ...","[Nagoya, Toyohashi, Okazaki, Ichinomiya, Seto,..."
豊,"bountiful, excellent, rich",Kun: ゆた.か、 とよ,On: ホウ、 ブ,"13 strokes. JLPT N2. Jōyō kanji, taught in...","[Toyohashi, Toyokawa, Toyota, Toyoake, Buzen, ..."
...,...,...,...,...,...
月,NaN,NaN,NaN,NaN,[Ōtsuki]
韮,NaN,NaN,NaN,NaN,[Nirasaki]
斐,NaN,NaN,NaN,NaN,[Kai]
笛,NaN,NaN,NaN,NaN,[Fuefuki]


In [ ]:
#Merge cities with 

In [131]:
t1 = pd.read_html("https://www.latlong.net/category/cities-111-15.html")[0]

In [134]:
t1['cityName'] = [x.split(',')[0] for x in t1['Place Name']]
t1 = t1.drop(columns='Place Name')

KeyError: 'Place Name'

In [153]:
t2 = cities.join(t1.set_index('cityName'), on='City (Special Ward)')
t2 = t2.dropna(subset='Longitude')
t2 = t2.drop(columns='Website')
t2

,City (Special Ward),Japanese,Prefecture,Population,Area (km2),Density (per km2),Founded,Latitude,Longitude
0,Nagoya,名古屋市,Aichi,2327557,326.45,6860.0,1889-10-01,35.183334,136.899994
38,Akita,秋田市,Akita,329287,905.67,364.0,1889-04-01,39.720009,140.102570
64,Funabashi,船橋市,Chiba,586762,85.64,6851.0,1937-04-01,35.694706,139.982620
118,Fukuoka,福岡市,Fukuoka,1588924,340.96,4515.0,1889-04-01,33.583332,130.399994
131,Kitakyushu,北九州市,Fukuoka,940978,487.71,2024.0,1963-02-10,33.883331,130.883331
193,Hiroshima,広島市,Hiroshima,1199391,905.13,1286.0,1889-04-01,34.383331,132.449997
207,Sapporo,札幌市,Hokkaido,1976257,1121.26,1763.0,1922-08-01,43.066666,141.350006
212,Obihiro,帯広市,Hokkaido,168188,618.94,272.0,1933-04-01,42.923901,143.196106
242,Kobe,神戸市,Hyōgo,1524601,552.23,2772.0,1889-04-01,34.689999,135.195557
253,Takarazuka,宝塚市,Hyōgo,222071,101.80,2181.0,1954-04-01,34.815723,135.361099


In [156]:
t2.reset_index().to_json('startcities.json', orient='index')